<a href="https://colab.research.google.com/github/EmidioLP/Freecodecamp/blob/main/Notebooks/Machine%20Learning%20with%20Python/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse notebook é baseado nas aulas disponibilizadas em: https://www.freecodecamp.org/portuguese/learn/machine-learning-with-python/

# Clustering

Clustering é uma téncnica de Machine Learning que envolve o agrupamento de pontos de dados. É um tipo de treinamento que chamamos de ***Não-Supervisionado***. Na teoria, pontos de dados que estão no mesmo grupo devem ter propriedades e/ou características similires, enquanto pontos de dados em grupos diferentes deveriam ter propriedades e/ou características altamente distintas. (https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68)

# Hidden Markov Model

"O Modelo Oculto de Markov é um conjunto finito de estados , cada um dos quais está associado a uma distribuição de probabilidade (geralmente multidimensional) []. As transições entre os estados são governadas por um conjunto de probabilidades chamadas probabilidades de transição." (http://jedlik.phy.bme.hu/~gerjanos/HMM/node4.html)

O Modelo Oculto de Markov trabalha com probabilidade para prever eventos futuros ou estados. Nesse módulo, irá se utilizar esse modelo para prever dados sobre o Clima. 

Essa seção é baseada no seguinte tutorial disponibilizado pelo próprio Tensorflow: https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel

# Dados

Primeiro vamos falar sobre o tipo de dados que usamos quando vamos trabalhar com o "modelo oculto de Markov".

Nas seções anteriores foram trabalhados grandes datasets de 100 ou mais diferentes entradas. Com o modelo de Markov, estamos interessados apenas na probabilidade de distribuição desses dados. 

Nós então acharemos essas probabilidades ocultas em grandes datasets ou até mesmo já tenhamos esses valores.

**States:** Em cada modelo de Markov há um número de estados finitos. Esses estados, como iremos trabalhar com uma base de dados de Clima, pode ser algo como "frio", "quente" ou "alto" e "baixo" ou "vermelho", "verde" e "azul". Esses estados estão "escondidos" dentro do modelo, o que significa que não podemos observa-los diretamente.

**Observations:** Cada estado tem uma saída particular ou associada a uma distribuição de probabilidade. Um exemplo pode ser como: Em um dia quente, Tim tem uma probabilidade de *80%* de estar feliz e *20%* de estar triste.

**Transitions:** Cada estado terá uma probibabilidade que define a possibilidade de transição para um estado diferente. Um exemplo pode ser o seguinte: Um dia frio tem *30%* de chance de ser seguido por um dia quente e *70%* de chance de ser seguido por outro dia frio.

Para a criação de um modelo oculto de Markov precisamos de: 


*   States (Estados)
*   Observation Distribution (Distribuição de observação)
*   Transition Distribution (Distribuição de transição)

Para o propósito desse módulo, é assumido que nós já temos essa informação disponível enquanto tentamos prever o clima em um dado dia.





# Imports e Setup

In [1]:
%tensorflow_version 2.x #Necessária apenas caso trabalhe-se com um notebook.

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import tensorflow_probability as tfp # Modulo do tensorflow para modelos que utilizam probabilidade
import tensorflow as tf

# Modelo de Clima

Foi retirado diretamente da documentação do Tensorflow
(https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/HiddenMarkovModel).

Criaremos um simples modelo de sistema de clima e vamos tentar prever a temperatura em cada dia com as seguintes informações:


1.   Dias frios são sempre codados como 0 e dias quentes com 1.
2.   O primeiro dia na nossa sequência tem 80% de chance de ser frio.
3.   Um dia frio tem 30% de chance de ser seguido por um dia quente.
4.   Um dia quente tem 20% de chance de ser seguido por um dia frio.
5.   Em cada dia a temperatura é normalmente distribuida com média e desvio padrão 0 e 5 em um dia frio e média e desvio padrão 15 e 10 em um dia quente. 

Ness exemplo, em um dia quente a temperatura é 15 e varia de 5 a 25.



In [5]:
tfd = tfp.distributions #Variável encurtada para usar futuramente
initial_distribution = tfd.Categorical(probs=[0.8, 0.2]) #Referência ao ponto 2 logo acima
transition_distribution = tfd.Categorical(probs=[[0.7, 0.3],
                                                  [0.2, 0.8]]) #Referência aos pontos 3 e 4 acima
observation_distribution = tfd.Normal(loc=[0., 15.], scale = [5., 10.]) #Referência ao ponto 5 acima

#O argumento loc represente a média e o scale é o desvio padrão

Agora que foram criadas as variáveis para modelagem do sistema é hora da criação do Modelo Oculto de Markov.

In [7]:
model = tfd.HiddenMarkovModel(
    initial_distribution = initial_distribution,
    transition_distribution = transition_distribution,
    observation_distribution = observation_distribution,
    num_steps = 7)

A variável "num_steps" represente o número de dias que queremos prever. Nesse caso queremos prever o clima para a semana, por isso 7. 

Para gerar as **temperaturas esperadas** em cada dia podemos fazer o seguinte.

In [8]:
mean = model.mean()
with tf.compat.v1.Session() as sess:
  print(mean.numpy())

[2.9999998 5.9999995 7.4999995 8.25      8.625001  8.812501  8.90625  ]
